# Introduction to LangChain

## Setup
#### Follow [README](https://github.com/tirtho/open-ai/blob/main/README.md) and perform setup before running the notebooks

Reference : 
- [Azure Open AI](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/overview)
- [LangChain home page](https://python.langchain.com/docs/get_started/introduction.html)

#### Load the API key and relevant Python libaries.

In [1]:
import openai
import sys

from azure_openai_setup import get_openai_client, get_config_from_os_env, get_chat_completion

THE_MODEL = 'gpt-4o'
endpoint, key, version = get_config_from_os_env()
#print(f"{endpoint}, {key}, {version}")


Got OPENAI API Key from environment variable
Connecting to Open AI returned status as True


## LangChain Simple Chatbot example
The below example makes direct LangChain library calls so it is better \
for us to understand this LangChain framework.\
In subsequent examples afterwards we will create and use a helper function.

In [4]:
from langchain.chat_models import AzureChatOpenAI

# The openai.<variables> are already filled up by the above 
# set_openai_config() helper function called from above cell.
# Check that function for more details

azureChatClient = AzureChatOpenAI(
            openai_api_key = key,
            openai_api_base = endpoint,
            openai_api_version = version,
            deployment_name = THE_MODEL,
            temperature=0
)

#### Asking bot to perform as task

In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

azureChatClient([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AttributeError: module 'openai' has no attribute 'error'

#### Generate completions for multiple sets of messaages

In [14]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = azureChatClient.generate(batch_messages)
for generation in result.generations:
    print(generation[0].message.content)

J'adore la programmation.
J'adore l'intelligence artificielle.


## Prompt Templates

The concept of templates make life so much easier!

#### Your Prompt Engineer creates the ChatPrompt object

and hands over the aChatPrompt to your Application Dev Engineer.

In [15]:
system_template = "You are a helpful assistance that translates to {target_language}."
aSystemPrompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{text}"
aHumanPrompt = HumanMessagePromptTemplate.from_template(human_template)

aChatPrompt = ChatPromptTemplate.from_messages([aSystemPrompt, aHumanPrompt])

#### Your App Dev Engineer receives aChatPrompt
Populates that prompt with user input and receives completion from the LLM model

In [16]:
theCompletion = azureChatClient(aChatPrompt.format_prompt(target_language="English Pirate", 
                                text="Hello Sir! Would you like to buy a bottle of rum from \
                                the Captain of the Black Pearl?").to_messages())

In [17]:
print(theCompletion.content)

Ahoy matey! Aye, I would be interested in purchasing a bottle of rum from the Captain of the Black Pearl. Pray tell, what be the price for such a fine bottle of grog?


## Output Parser
Helps you get the completion parsed in your desired format. Different types of output parsers are

- PydanticOutputParser
- CommaSeparatedListOutputParser
- Datetime
- Enum
- OutputFixingParser
- RetryOutputParser
- Structured Output Parser

Let's checkout the Structured Output Parser

#### Let's take this actual quote

In [18]:
actual_quote_struct = {
    'auto': True,
    'premium': '$200',
    'driver': 'TR BARARI',
    'vin': 'JH4DA9460LS000685',
    'liability':'$100000',
    'collision_deduct':'$250',
    'comprehensive_deduct':'$100',
    'personal_injury':True,
    'uninsured':True,
    'underinsured':True
}

#### Here is the example quote in an email

In [19]:
auto_insurance_quote_text = f"""
Dear Mr TR Barari, \

I would like to take this opportunity to congratulate you \
on being approved for your auto insurance from Reliable Insurance Inc. \
 \
Here are the details of the quote. \
 \
    Monthly premium is $200 \
    Driver Name is TR BARARI \
    Auto VIN Number is JH4DA9460LS000685 \
    Coverage for Liability Insurance is $100000 \
    Collision is covered with a per claim deductible of $250 \
    Comprehensive insurance is included with a per claim deductible of $100 \
    Personal_injury is covered \
    Includes full coverage against uninsured and underinsured motorist \
    Coverage Start Date 06/01/2023 \
 \
Please email back with your acceptance of this offer in the next 3 days \
for this to be effective. \
 \
Regards \
 \
John Doe \
Vice President \
Reliable Insurance Inc.\
1 Main Street, NY 12345 \
Ph: +1 (234) 567-9876
"""

#### The template from the Prompt Engineer with instructions to process quote emails

In [20]:
auto_insurance_quote_template = """
Extract from {text} based on the instructions in {format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=auto_insurance_quote_template)

#### Now let's use the LangChain output parser to parse insurance quotes

In [21]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

auto_schema = ResponseSchema(name="auto",
                             description="Was this an Auto Insurance Quote? Answer as True or False")
premium_schema = ResponseSchema(name="premium",
                             description="How much is the monthly premium?")
driver_schema = ResponseSchema(name="driver",
                             description="What is the name of the driver?")
vin_schema = ResponseSchema(name="vin",
                             description="What is the VIN Number of the auto?")
liability_schema = ResponseSchema(name="liability",
                             description="How much is the liability coverage?")
collision_deduct_schema = ResponseSchema(name="collision_deduct",
                             description="How much is the collision deductible per claim?")
comprehensive_deduct_schema = ResponseSchema(name="comprehensive_deduct",
                             description="How much is the comprehensive deductible per claim?")
uninsured_schema = ResponseSchema(name="uninsured",
                             description="Is uninsured motorist coverage included? Answer as True or False")
underinsured_schema = ResponseSchema(name="underinsured",
                             description="Is underinsured motorist coverage included? Answer as True or False")

insurance_quote_schemas = [auto_schema, premium_schema, driver_schema, 
                    vin_schema, liability_schema, collision_deduct_schema, 
                    comprehensive_deduct_schema, uninsured_schema, underinsured_schema]

insurance_quote_parser = StructuredOutputParser.from_response_schemas(insurance_quote_schemas)

auto_insurance_quote_format_instructions = insurance_quote_parser.get_format_instructions()

#### Developer creates the message, sends it to the model and gets the response back

In [22]:
messages = prompt.format_messages(text=auto_insurance_quote_text,
                                 format_instructions=auto_insurance_quote_format_instructions)
response_from_model = azureChatClient(messages)

#### Use the LangChain parser to parse the output

In [23]:
output_quote_struct = insurance_quote_parser.parse(response_from_model.content)

In [24]:
output_quote_struct

{'auto': True,
 'premium': '$200',
 'driver': 'TR BARARI',
 'vin': 'JH4DA9460LS000685',
 'liability': '$100000',
 'collision_deduct': '$250',
 'comprehensive_deduct': '$100',
 'uninsured': True,
 'underinsured': True}

#### Test if the expected quote is same as the quote extracted by the model

In [25]:
# Check if the quoted $ is same in both

if output_quote_struct.get('premium') == actual_quote_struct.get('premium'):
    print("SUCCESS: The expected quote is same as the output quote extracted by the Azure OpenAI model")
else:
    print("FAILED: The expected and output quote are not same")


SUCCESS: The expected quote is same as the output quote extracted by the Azure OpenAI model
